# Tratamento de dados

In [ ]:
%pip install pandas
%pip install numpy
%pip install missingno
%pip install plotly.express

import pandas as pd
import numpy as np
import missingno as msno
import plotly.express as px

In [2]:
lista = ['1_bases_originais/baseoriginal.csv']

In [3]:
dados = pd.DataFrame(pd.read_csv(lista[0]))
  
for i in range(1,len(lista)): 
    data = pd.read_csv(lista[i]) 
    df = pd.DataFrame(data) 
    df['detalhe']=lista[i][20:]
    dados = pd.concat([dados,df],axis=0)

### Editando os nomes das colunas

In [4]:
dados.columns = dados.columns.str.lower()
dados.columns = dados.columns.str.replace('\W', '')

### Substituindo caracteres especiais de toda a base de dados

In [5]:
dados = dados.replace({'á':'a','ã':'a', 'â':'a', 'í':'i',
                '\W':' ',
                "á":"a",
                "é":"e",
                "í":"i",
                "ó":"o",
                "ú":"u",
                "ã":"a",
                "õ":"o",
                "á":"a",
                "é":"e",
                "í":"i",
                "ó":"o",
                "ú":"u",
                "ã":"a",
                "õ":"o",
                "ê":"e",
                "â":"a",
                "ô":"o",
                "ê":"e",
                "â":"a",
                "ô":"o",
                "\\\\n": ''              
              
              }, regex=True)

In [6]:
dados['vendas'].replace({' ':''}, regex=True, inplace=True)

### Dados missing


In [7]:
dados.isnull().sum()

rank              0
titulo            6
vendas            0
lancamento        0
desenvolvedora    2
dtype: int64

In [ ]:
%pip install missingno
import missingno as msno

### Eu escolhi dropar todas as linhas onde o título é vazio, pois eu reparei que nas linhas onde o título é vazio, todos os outros dados também são corrompidos. Isto deve-se á má formatação de tabela do site wikipedia, fazendo com que alguns dados fossem pegos de células erradas.

In [9]:
dados.dropna(axis=0,how='any', subset='titulo', inplace=True)

### Trocando dados nulos por "missing" na tabela

In [10]:
dados.fillna(-3, inplace=True)
dados.loc[dados.desenvolvedora==-3, 'desenvolvedora']='missing'

In [11]:
dados.index.value_counts()
dados.reset_index(inplace=True, drop=True).index.value_counts()

AttributeError: 'NoneType' object has no attribute 'index'

In [ ]:
dados

,rank,titulo,vendas,lancamento,desenvolvedora
0,1,Minecraft,238000000,18 de novembro de 2011,Mojang
1,2,Grand Theft Auto V,175000000,17 de setembro de 2013,Rockstar North
2,3,Tetris,170000000,6 de junho de 1984,missing
3,4,Wii Sports,82900000,19 de novembro de 2006,Nintendo EAD
4,5,PlayerUnknown s Battlegrounds,75000000,20 de dezembro de 2017,PUBG Studios
5,6,Mario Kart 8 Deluxe,60460000,29 de maio de 2014,Nintendo EAD
6,7,Super Mario Bros,58000000,13 de setembro de 1985,Nintendo
7,8,Overwatch,50000000,24 de maio de 2016,Blizzard Entertainment
8,10,Pokemon Red Green Blue,47520000,27 de fevereiro de 1996,Game Freak
9,11,Terraria,44500000,16 de maio de 2011,missing


# Criando e salvando a base tratada em um banco de dados

In [ ]:
%pip install sqlalchemy

import sqlalchemy as db
from sqlalchemy import create_engine, text

In [13]:
engine = db.create_engine("sqlite:///bancojogos.db", echo=True)

In [14]:
conn = engine.connect() 

In [15]:
metadata = db.MetaData()

In [ ]:
dados.to_sql('DBJogos', con=engine, if_exists='replace', index=False)

In [ ]:
query = text("SELECT * FROM DBJogos")
result = conn.execute(query)
result.fetchall()